In [ ]:
import random
import spacy
from spacy.util import minibatch, compounding
from spacy.training import Example
import csv

# Function to parse entities from the string representation
def parse_entities(entity_str):
    entities = []
    entity_list = entity_str.split(", ")
    for entity in entity_list:
        start_end, label = entity.split(":")
        start, end = map(int, start_end.split("-"))
        entities.append((start, end, label))
    return entities

# Load the training data from CSV
TRAIN_DATA = []
with open("/content/ner_training_data (1) (1).csv", "r", encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        sentence = row["sentences"]
        entities_str = row["entities"]
        entities = parse_entities(entities_str)
        TRAIN_DATA.append((sentence, {"entities": entities}))

nlp = spacy.blank("en")
ner = nlp.add_pipe('ner')
ner.add_label("FOOD")

# Fix the random seed for reproducibility
spacy.util.fix_random_seed(0)

# Getting names of other pipes to disable them during training
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

# Only train NER
with nlp.disable_pipes(*unaffected_pipes):
    optimizer = nlp.begin_training()
    for i in range(100):  # Let's train for a bit longer
        random.shuffle(TRAIN_DATA)
        losses = {}

        # Batch the examples and iterate over them
        for batch in minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001)):
            for text, annotations in batch:
                # Create an Example object
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                # Update the model
                nlp.update([example], drop=0.1, sgd=optimizer, losses=losses)
        print(f"Losses at iteration {i} - {losses}")

# Test the trained model on the test_sentences
test_sentences = [
    "Do you serve burgers with cheese here?",
    "I want misal pav with a side of roasted peanuts for added crunch?",
    "One order of Tandoori Chicken with tangy tamarind chutney",
    "I will Tandoori Chicken with a butter naan",
    "I want to enjoy the flavors of Tandoori Chicken with dal makhani",
    "I want to enjoy the Pani Puri with a side of spicy chutney",
    "Can I have some Pani Puri with extra sweet tamarind water, please?",
    "May I have a serving of Pani Puri with some sev on top?",
    "Please serve me a plate of Veg Pulao with raita and papad",
    "I'll have a Chicken Shawarma",
    "I'll have a Veg Biryani",
    "Can I get a Caesar Salad?",
    "One plate of Chana Masala with Bhature, please.",
    "I'd love to try your Paneer Tikka",
    "I like to savor some Palak Paneer with garlic naan and a side of raita.",
    "I like to savor some Palak Paneer with garlic naan and a side of raita.",
    "Please bring me a plate of Aloo Paratha with mango pickle and a dollop of butter.",
    "I would like some Hyderabadi Biryani with Mirchi ka Salan and cucumber salad.",
    "Can I get a plate of Rajma Chawal with papad and some onion salad?",
    "I want to enjoy the flavors of Malai Kofta with butter naan and a refreshing glass of Lassi.",
    "I want to try your Gulab Jamun with a scoop of vanilla ice cream.",
    "make me a Vada Pav extra spicy and serve it with green chilies?",
    "I like to try your Kaju Katli with a touch of saffron.",
    "I want Rasgulla with some rose syrup.",
    "I like Vada Pav, a popular Maharashtrian street food.",
    "Please bring me a plate of Misal Pav, a spicy and flavorful Maharashtrian dish.",
    "I craving for some Puran Poli, a delicious sweet flatbread from Maharashtra.",
    "Can I get a serving of Pav Bhaji, a mouthwatering Maharashtrian fast food?",
    "Please serve me a bowl of Bharli Vangi, a stuffed eggplant delicacy from Maharashtra.",
    "One plate of Sabudana Khichdi  a popular fasting dish in Maharashtra, please.",
    "I want to enjoy the flavors of Kothimbir Vadi  a savory cilantro snack from Maharashtra.",
    "Can you make my Puran Poli extra sweet and serve it with a dollop of ghee?",
    "I have a plate of Bombil Fry, a crispy and delicious Maharashtrian fish preparation.",
    "Please bring me a bowl of Solkadhi, a traditional Maharashtrian drink made with coconut milk and kokum.",
    "I am in mood for some Kanda Bhaji a popular onion fritter from Maharashtra.",
    "I want to enjoy the flavors of Shrikhand a popular sweet dish from Maharashtra.",
    "Please serve me a plate of Kanda Bhaji a popular Maharashtrian snack.",
    "I want to enjoy the flavors of Batata Vada, a spicy potato fritter.",
    "craving for some Dhokla a famous Gujarati snack.",
    "Can I get a serving of Undhiyu a traditional Gujarati mixed vegetable dish?",
    "I like to order some Vegetable Manchurian a popular Indo-Chinese dish.",
    "One plate of Kutchi Dabeli a famous Gujarati street food, please.",
    "Please bring me a plate of Rava Idli with coconut chutney",
    "One plate of Mini Idli with sambar please.",
    "Can you make my Poha with extra peanuts?",
    "I want to try your Vangi Poha with eggplant",
    "Poha Dhokla"
]


for test_text in test_sentences:
    doc = nlp(test_text)
    print("Entities in '%s'" % test_text)
    for ent in doc.ents:
        print(ent.label_, ent.text)
    print()


Losses at iteration 0 - {'ner': 166.9391132237643}
Losses at iteration 1 - {'ner': 58.51907823615757}
Losses at iteration 2 - {'ner': 34.750369738029434}
Losses at iteration 3 - {'ner': 13.750711254394162}
Losses at iteration 4 - {'ner': 7.866005869059506}
Losses at iteration 5 - {'ner': 8.565902253636505}
Losses at iteration 6 - {'ner': 6.086806871614725}
Losses at iteration 7 - {'ner': 5.2512152367694345}
Losses at iteration 8 - {'ner': 3.999107141467077}
Losses at iteration 9 - {'ner': 0.019768036103673125}
Losses at iteration 10 - {'ner': 4.1635228054129946e-05}
Losses at iteration 11 - {'ner': 0.0006545236688401502}
Losses at iteration 12 - {'ner': 8.404285959006795e-05}
Losses at iteration 13 - {'ner': 0.00011305284492029723}
Losses at iteration 14 - {'ner': 4.509196211399517e-05}
Losses at iteration 15 - {'ner': 7.752753352627822e-06}
Losses at iteration 16 - {'ner': 3.938125385489152e-07}
Losses at iteration 17 - {'ner': 3.326304697962322e-07}
Losses at iteration 18 - {'ner': 7